In [1]:
from PIL import Image
import numpy as np

image = Image.open("data/image/10165.jpg")
image
np_image = np.array(image)
print(np_image.shape)
#print(np_image)
print(np.sum(np_image)/np_image.shape[0]/np_image.shape[1]/np_image.shape[2])

(3000, 4000, 3)
72.27082336111111


In [2]:
#import csv
#filename = open("FloodNet_split_train_valid_test.csv", 'r')

import pandas as pd

df = pd.read_csv("FloodNet_split_train_valid_test.csv", sep=',', header=None, names=["Column1", "Column2", "Column3"])
df1 = df[df["Column3"]=='train']
df1["Column1"].map(lambda x: "data/" + x)#.iloc[1]

list(zip(df1["Column1"], df1["Column2"]))  
df1['Column1']

for entry in list(df['Column1']):
    filepath = 'Data/Image/' + entry
    image = Image.open(filepath)
    np_image = np.array(image)
    if np_image.shape[0] != 3000:
        print(entry, np_image.shape)


10828.jpg (3072, 4592, 3)
10836.jpg (3072, 4592, 3)
10837.jpg (3072, 4592, 3)
10840.jpg (3072, 4592, 3)
6585.jpg (3072, 4592, 3)
6586.jpg (3072, 4592, 3)
6589.jpg (3072, 4592, 3)
6595.jpg (3072, 4592, 3)
6600.jpg (3072, 4592, 3)
6614.jpg (3072, 4592, 3)
6615.jpg (3072, 4592, 3)
6618.jpg (3072, 4592, 3)
6623.jpg (3072, 4592, 3)
6628.jpg (3072, 4592, 3)
6692.jpg (3072, 4592, 3)
6693.jpg (3072, 4592, 3)
6703.jpg (3072, 4592, 3)
6704.jpg (3072, 4592, 3)
6706.jpg (3072, 4592, 3)
6707.jpg (3072, 4592, 3)
6716.jpg (3072, 4592, 3)
6994.jpg (3072, 4592, 3)
6997.jpg (3072, 4592, 3)
7240.jpg (3072, 4592, 3)
7243.jpg (3072, 4592, 3)
7261.jpg (3072, 4592, 3)
7266.jpg (3072, 4592, 3)
7267.jpg (3072, 4592, 3)
7273.jpg (3072, 4592, 3)
7282.jpg (3072, 4592, 3)
7289.jpg (3072, 4592, 3)
7298.jpg (3072, 4592, 3)
7317.jpg (3072, 4592, 3)
7321.jpg (3072, 4592, 3)
7325.jpg (3072, 4592, 3)
7330.jpg (3072, 4592, 3)
7331.jpg (3072, 4592, 3)
7332.jpg (3072, 4592, 3)
7357.jpg (3072, 4592, 3)
7362.jpg (3072, 4592,

In [3]:
from torch.utils.data import Dataset

from PIL import Image

import os
import glob

class FloodData(Dataset):

    # mapping between label class names and indices
    LABEL_CLASSES = {
      'background': 		  0,
      'building-flooded': 			    1,
      'building-non-flooded': 	  2,
      'road-flooded': 				      3,
      'road-non-flooded': 			    4,
      'water': 			    5,
      'tree':   6,
      'vehicle': 				    7,
      'pool': 				    8,
      'grass': 			  9
    }

    
    

    def __init__(self, transforms=None, split='train'):
        
        self.transforms = transforms
        
        SPLIT = pd.read_csv("FloodNet_split_train_valid_test.csv", sep=',', header=None, names=["Column1", "Column2", "Column3"])
        SPLIT["Column1"] = SPLIT["Column1"].map(lambda x: "Data/image/" + x)
        SPLIT["Column2"] = SPLIT["Column2"].map(lambda x: "Data/mask/" + x)
        
        splitted_set = SPLIT[SPLIT["Column3"]==split]
        
        # prepare data
        self.data = list(zip(splitted_set["Column1"], splitted_set["Column2"]))                                  # list of tuples of (image path, label class)
        """
        images = np.empty((len(self.data)*3000,len(self.data)*4000,3))
        
        for i in range(len(self.data)):
            images[i] = np.array()
            Image.open()
        """ 
            
    #TODO: please provide the remaining functions required for the torch.utils.data.Dataset class.
    def __len__(self):
        return len(self.data)


    def __getitem__(self, x):
        imgName, labelsName = self.data[x]

        img = np.array(Image.open(imgName))
        labels = np.array(Image.open(labelsName))
        if self.transforms is not None:
            transformed = self.transforms(image=img, mask=labels)
            img = transformed['image']
            labels = transformed['mask']
        else:
            img, labels = img[:3000, :4000,:], labels[:3000, :4000]
        
        img, labels = torch.tensor(img, dtype=torch.int32), torch.tensor(labels)
        return img, labels


In [4]:
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

# source of code in this cell:
# https://www.binarystudy.com/2021/04/how-to-calculate-mean-standard-deviation-images-pytorch.html

train_not_transformed_set = FloodData(transforms = None, split = 'train')
train_not_transformed_loader = DataLoader(train_not_transformed_set, batch_size = 16)

def batch_mean_and_sd(loader):
    count = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)
    
    for images, _  in tqdm(loader):
        b, h, w, c = images.shape #batch, height, width, color
        nb_pixels = b*h*w
        
        sum_ = torch.sum(images, dim = [0,1,2])
        sum_of_square = torch.sum(torch.square(images), dim = [0,1,2])
        
        fst_moment = (count*fst_moment+sum_)/(count+nb_pixels)
        snd_moment = (count*snd_moment+sum_of_square)/(count+nb_pixels)

        count += nb_pixels
        
    mean, std = fst_moment, torch.sqrt(snd_moment-fst_moment**2)
    return mean, std

mean, std = batch_mean_and_sd(train_not_transformed_loader)
print(mean, std)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [04:28<00:00, 14.91s/it]


tensor([106.5385, 116.1601,  87.6059]) tensor([53.1837, 49.5205, 53.5828])


In [5]:
import albumentations as A
import cv2


transform_train = A.Compose([
    A.RandomSizedCrop(min_max_height = [1000, 2500], height=3000, width=4000),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(p=0.5),
    #A.Blur(blur_limit = 3),
    A.RandomRotate90(),
    #A.OpticalDistortion(),
    #A.GridDistortion(),
    A.Normalize(mean = mean, std = std),
])

transform_val = A.Compose([
    A.RandomSizedCrop(min_max_height = [1000, 2500], height=3000, width=4000),
    A.Normalize(mean = mean, std = std),
])

In [6]:
train_dataset = FloodData(transforms = transform_train, split = 'train')
val_dataset = FloodData(transforms = transform_val, split = 'valid')
test_dataset = FloodData(transforms = transform_val, split = 'test')

In [7]:
train_loader = DataLoader(train_dataset, batch_size = 16)
next(iter(train_dataset))[0]

tensor([[[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]],

        [[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]],

        [[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]],

        ...,

        [[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]],

        [[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]],

        [[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1],
         ...,
         [-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]]], dtype=torch.int32)